# DNN

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

2023-11-18 01:42:42.030051: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-18 01:42:42.134526: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-18 01:42:42.662546: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/onnxruntime-linux-x64-1.6.0:/opt/xilinx/xrt/lib::/home/chms/miniconda3/envs/

## 데이터 가져오기

In [2]:
train_set = pd.read_csv('train_set.csv')
train_set = train_set[train_set['ECLO'] <= 25]
test_set = pd.read_csv('test_set.csv')

## train test valid 분리

In [3]:
train_x, test_x, train_y, test_y = train_test_split(train_set[['요일', '기상상태', '도로형태', '노면상태', '사고유형']], train_set['ECLO'], test_size=0.2)

In [4]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2)

## tensorflow model 정의

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(5)),
    tf.keras.layers.Dense(5, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(10, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(50, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(50, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(10, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(1)]
)

2023-11-18 01:42:43.672005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-18 01:42:44.244117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 537 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:af:00.0, compute capability: 8.6


In [6]:
checkpoint_filepath = "logs/DNN/"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
)

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.MeanSquaredLogarithmicError(), metrics=['mae'])
model.fit(train_x, train_y, epochs=100, validation_data=(valid_x, valid_y),
          callbacks=[
                        checkpoint_callback,
                        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3),
                    ])

Epoch 1/100


2023-11-18 01:42:45.901037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-18 01:42:45.903859: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f8f45949110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-18 01:42:45.903876: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-11-18 01:42:45.907726: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-18 01:42:46.055506: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


791/791 [==============================] - 8s 7ms/step - loss: 0.2395 - mae: 2.1752 - val_loss: 0.2132 - val_mae: 1.9890
Epoch 2/100
791/791 [==============================] - 5s 6ms/step - loss: 0.2009 - mae: 2.0575 - val_loss: 0.2044 - val_mae: 2.0373
Epoch 3/100
791/791 [==============================] - 5s 6ms/step - loss: 0.2007 - mae: 2.0571 - val_loss: 0.2060 - val_mae: 2.0689
Epoch 4/100
791/791 [==============================] - 5s 6ms/step - loss: 0.2006 - mae: 2.0573 - val_loss: 0.2048 - val_mae: 2.0551
Epoch 5/100
791/791 [==============================] - 5s 6ms/step - loss: 0.2006 - mae: 2.0579 - val_loss: 0.2048 - val_mae: 2.0267


In [8]:
np.sqrt(mean_squared_log_error(test_y, model.predict(test_x)))

248/248 [==============================] - 1s 2ms/step


0.44385643106571354

## 결과 저장

In [9]:
ECLOs = model.predict(test_set[['요일', '기상상태', '도로형태', '노면상태', '사고유형']])
ans = pd.read_csv('open/sample_submission.csv')
ans['ECLO'] = ECLOs
ans.to_csv('ans_dnn.csv', index=False)

343/343 [==============================] - 1s 2ms/step
